MIT License

Copyright (c) 2021 Taiki Miyagawa and Akinori F. Ebihara

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# Define Train/Valid/Test Splitting (text files)
DATADIR/UCF101/labels/ucfTrainValidTestlist/ will be made.

In [22]:
from glob import glob
import os, shutil
from copy import copy
import statistics
import matplotlib.pyplot as plt
import numpy as np

## Preliminaries

In [ ]:
DATADIR = "Define this first, E.g., /data/t-miyagawa"
splitnum = 1 # official splitting number. 1, 2, or 3

In [23]:
# Get videodir and numf
datadir = "{}/UCF101png".format(DATADIR)
classdir = sorted(glob(datadir + "/*"))
classdir = [i + "/" for i in classdir]
classnames = [i[i.rfind("UCF101png/") + 10 : -1] for i in classdir]
videodir = {
    k : 
    sorted(glob([v for v in classdir if v.find("/" + k + "/") != -1][0] + "/*"))
    for k in classnames}

numf = dict()
for k in classnames:
    v = videodir[k]
    numf[k] = [len(glob(_video + "/*.png")) for _video in v]

# Extend dict to list
numf_concat = []
for k in classnames:
    v = numf[k]
    numf_concat.extend(v)

videodir_concat = []
for k in classnames:
    v = videodir[k]
    videodir_concat.extend(v)
    
# Classwise num of frames
numf_classwise = []
for k in classnames:
    v = numf[k]
    v = sum(v)
    numf_classwise.append(v)
    
# Classwise num of videos
numv_classwise = []
for k in classnames:
    v = videodir[k]
    v = len(v)
    numv_classwise.append(v)
    
# Classwise num of unique videos (groups)
numuv_classwise = []
for k in classnames:
    v = videodir[k][-1]
    numuv_classwise.append(int(v[v.rfind("_g") + 2: v.rfind("_c")]))
    

# """
# Returns:
#     classnames: List. Len = Num of classes. Names of classes in alphabetical order.
#
#     videodir: Dict. Paths to video directories. Each values (paths) are in alphabetical order of video names.
#     numf: Dict. Num of frames for each videos. Each values (integers) are in alphabetical order of video names.
#
#     numf_concat: List. Len = Num of total videos. Order is the same as `videoddir_concat`.
#     videodir_concat: List. Len = Num of total videos. Order is the same as `numf_concat`.
#
#     numf_classwise: List. Len = Num of classes. The classwise numbers of frames in alphabetical order of class names.
#     numv_classwise: List. Len = Num of classes. The classwise numbers of videos in alphabetical order of class names.
#     numuv_classwise: List. Len = Num of classes. The classwise numbers of unique videos (groups) in alphabetical order of class names.
# """

## Save train/valid/test/list0n.txt
Change n = 'splitnum'. 

In [99]:
# Define file names
#################################################
trtxt = "{}/UCF101/labels/ucfTrainTestlist/trainlist0{}.txt".format(DATADIR, splitnum)
tetxt = "{}/UCF101/labels/ucfTrainTestlist/testlist0{}.txt".format(DATADIR, splitnum)

newtrtxt = trtxt.replace("ucfTrainTestlist", "ucfTrainValidTestlist")
newtetxt = tetxt.replace("ucfTrainTestlist", "ucfTrainValidTestlist")
newvatxt = trtxt.replace("ucfTrainTestlist", "ucfTrainValidTestlist").replace("trainlist", "validlist")

if not os.path.exists("{}/UCF101/labels/ucfTrainValidTestlist".format(DATADIR)):
    os.makedirs("{}/UCF101/labels/ucfTrainValidTestlist".format(DATADIR))

# Load label text files
#################################################
with open(trtxt) as f:
    clips = f.readlines() # list
trclips = ["/" + clip[:clip.rfind(".avi")] for clip in clips]
trgroups = ["/" + clip[:clip.rfind(".avi") - 4] for clip in clips]
    # e.g., '/YoYo/v_YoYo_g25'

with open(tetxt) as f:
    clips = f.readlines() # list
teclips = ["/" + clip[:clip.rfind(".avi")] for clip in clips]
tegroups = ["/" + clip[:clip.rfind(".avi") - 4] for clip in clips]
    # e.g., '/YoYo/v_YoYo_g25'

# E.g., trclips =
# ['/ApplyEyeMakeup/v_ApplyEyeMakeup_g08',
#  '/ApplyEyeMakeup/v_ApplyEyeMakeup_g08',
#  '/ApplyEyeMakeup/v_ApplyEyeMakeup_g08',
#  ...]

# E.g., trgroups =
# ['/ApplyEyeMakeup/v_ApplyEyeMakeup_g08',
#  '/ApplyEyeMakeup/v_ApplyEyeMakeup_g08',
#  '/ApplyEyeMakeup/v_ApplyEyeMakeup_g08',
#  ...]


# Extract unique-group list
#################################################
trgroupsu = list(np.unique(trgroups))
tegroupsu = list(np.unique(tegroups))
    # E.g.,
    # tegroupsu =
    # ['/ApplyEyeMakeup/v_ApplyEyeMakeup_g01',
    #  '/ApplyEyeMakeup/v_ApplyEyeMakeup_g02',
    #  '/ApplyEyeMakeup/v_ApplyEyeMakeup_g03',
    #   ...
    #  '/YoYo/v_YoYo_g05', '/YoYo/v_YoYo_g06', '/YoYo/v_YoYo_g07']

    
# New train/valid/test split
#################################################
newtegroupsu = copy(tegroupsu)
newtrgroupsu = []
newvagroupsu = []
for cls in classnames:
    _tmp = [v for v in trgroupsu if v.find("/" + cls + "/") != -1]
    newtrgroupsu.extend(_tmp[:-2])
    newvagroupsu.extend(_tmp[-2:])

for v in newvagroupsu:
    assert not v in newtrgroupsu
    
#     E.g.,
#     newtegroupsu =
#     ['/ApplyEyeMakeup/v_ApplyEyeMakeup_g01',
#      '/ApplyEyeMakeup/v_ApplyEyeMakeup_g02',
#      '/ApplyEyeMakeup/v_ApplyEyeMakeup_g03',
#       ...
#      '/YoYo/v_YoYo_g06', '/YoYo/v_YoYo_g07']


# Fetch clip number
#################################################
newteclips = copy(teclips)
newtrclips = []
newvaclips = []

for v in newtrgroupsu:
    newtrclips.extend([i for i in trclips if v in i])
for v in newvagroupsu:
    newvaclips.extend([i for i in trclips if v in i])
    
    
# Save train/valid/testlist01.txt 
# E.g., "DATADIR/UCF101/labels/ucfTrainValidTestlist/validlist01.txt"
#################################################
with open(newtrtxt, mode="w") as f:
    l = len(newtrclips)
    for i, v in enumerate(newtrclips):
        f.write(v + " \n")

with open(newvatxt, mode="w") as f:
    l = len(newvaclips)
    for i, v in enumerate(newvaclips):
        f.write(v + " \n")

with open(newtetxt, mode="w") as f:
    l = len(newteclips)
    for i, v in enumerate(newteclips):
        f.write(v + " \n")
        
# Assert
with open(newtrtxt, mode="r") as f:
    a = f.readlines()
with open(newvatxt, mode="r") as f:
    b = f.readlines()
with open(newtetxt, mode="r") as f:
    c = f.readlines()
for i in a:
    assert not i in b
    assert not i in c
for i in b:
    assert not i in a
    assert not i in c
for i in c:
    assert not i in a
    assert not i in b